## Module 3: Radial Basis Function (RBF) Interpolation

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'axes.labelsize': 18})
import numpy as np

In [ ]:
def phi_invquad(r,l): return 1./(1 + (r/l)**2)
def phi_invmultiquad(r,l): return 1./np.sqrt(1 + (r/l)**2)
def phi_multiquad(r,l): return np.sqrt(1 + (r/l)**2)
def phi_linear(r,l): return r/l
phis = [ phi_invquad, phi_invmultiquad, phi_multiquad, phi_linear]

In [ ]:
rr = np.linspace(0,20,101)
for phi in phis:
    plt.plot(rr, phi(rr,10), label=phi.__name__)
plt.ylim(0,1.5)
plt.xlabel(r'$r$'); plt.ylabel(r'$\phi$')
plt.legend()

### Radial basis function interpolation in 1d

In [ ]:
N_1d = 10                         ### Number of samples
a,b = -4,4                        ### Interval of interest
xi = np.linspace(a,b,N_1d+1)      ### Uniform sample locs
xx = np.linspace(a,b,501)         ### Sampling of x, for plotting

def f_1d(x): return np.sin(x)  ### Target function
def f_1d(x): return x > 0
fi = f_1d(xi)                     ### Sample values 

In [ ]:
plt.plot(xx,f_1d(xx))
plt.plot(xi,fi,'ok')

In [ ]:
def dist_1d(x1, x2): return np.abs(x1-x2)

In [ ]:
phi,l = phi_invquad,.1  ### RPD
for x in xi:
    plt.plot(xx, phi(dist_1d(x,xx),l))
plt.plot(xi,0.*xi,'ok')

In [ ]:
def interpolation_matrix_1d(xi,phi,l):
    N = len(xi)-1
    A = np.zeros((N+1,N+1))
    for i in range(N+1):
        A[i,:] = phi(dist_1d(xi[i],xi),l)
    return A
A = interpolation_matrix_1d(xi,phi,l)
plt.imshow(A, interpolation='none')

In [ ]:
coeffs = np.linalg.solve(A,fi)
coeffs

In [ ]:
def reconstruct(xx):
    out = np.zeros(xx.shape)
    for i,x in enumerate(xi):
         out += coeffs[i] * phi(dist_1d(x,xx),l)
    return out

In [ ]:
plt.plot(xx, reconstruct(xx), label='RBF')
plt.plot(xx, f_1d(xx), label='f exact')
plt.plot(xi, fi, 'ok', label='samples')
plt.legend()

### Radial basis function interpolation in 2d

In [ ]:
N_2d = 40
N_x,N_y = 9,9
xmin,xmax,ymin,ymax = -4,4,-4,4

def f1_2d(x,y): return y**2/(1.+x**2)
def f2_2d(x,y): return x**2 + y**2
def dist_2d(x1,y1, x2,y2):
    return np.sqrt((x1-x2)**2+(y1-y2)**2) ### Metric

xx = np.linspace(xmin,xmax,41)  ### Sampling of x, for plotting
yy = np.linspace(ymin,ymax,41)  ### Sampling of y
xx,yy = np.meshgrid(xx, yy)     ### Tensor-product grid 

In [ ]:
phi, l = phi_invquad, 1.

from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111,projection='3d')
pp = phi(dist_2d(xx,yy, 0,0),l)
ax.plot_surface(xx,yy,pp,rstride=1,cstride=1,linewidth=0,cmap=cm.coolwarm)

In [ ]:
random = False
if random:
    ### Sample grid in 2d - random
    xi,yi = np.random.random((2,N_2d))
    xi = (xmax-xmin)*xi + xmin
    yi = (ymax-ymin)*yi + ymin
else:
    ### Sample grid in 2d - structured
    x, y = np.linspace(-4,4,9), np.linspace(-4,4,9)
    xi,yi = np.meshgrid(x, y)
xi, yi = xi.reshape(-1), yi.reshape(-1)

In [ ]:
### Plot the grid
plt.plot(xi,yi,'ok')
plt.xlim(xmin,xmax); plt.ylim(ymin,ymax)

**(c) Plot $f(x,y)$, together with the sample locations to have an idea of what kind of data is available to the interpolation method. Change from a uniform grid to a grid with randomly distributed points.**

In [ ]:
fi = f1_2d(xi,yi)

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111,projection='3d')
ff = f1_2d(xx,yy)
ax.plot_surface(xx,yy,ff,rstride=1,cstride=1,linewidth=0,cmap=cm.coolwarm)
ax.plot(xi,yi,fi,'ok')

#### Interpolation conditions

In [ ]:
def interpolation_matrix_2d(xi,yi,phi,l):
    N = len(xi)-1
    A = np.zeros((N+1,N+1))
    for i in range(N+1):
        A[i,:] = phi(dist_2d(xi[i],yi[i],xi,yi),l)
    return A
A = interpolation_matrix_2d(xi,yi,phi,l)
plt.imshow(A, interpolation='none')

In [ ]:
coeffs = np.linalg.solve(A,fi)
coeffs, fi

#### Reconstruction

In [ ]:
def reconstruct_2d(xx,yy):
    out = np.zeros(xx.shape)
    for i,(x,y) in enumerate(zip(xi,yi)):
         out += coeffs[i] * phi(dist_2d(x,y,xx,yy),l)
    return out

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111,projection='3d')
zz = reconstruct_2d(xx,yy)
ax.plot_surface(xx,yy,zz,rstride=1,cstride=1,linewidth=0,cmap=cm.coolwarm)
ax.plot(xi, yi, fi, 'ok', label='samples')

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(221)
cs = plt.contourf(xx,yy,ff,31)
plt.plot(xi, yi, 'ok')
plt.title('Original')
plt.subplot(222)
### Use the same levels in both plots with cs.levels
plt.contourf(xx,yy,zz,cs.levels)
plt.plot(xi, yi, 'ok')
plt.title('RBF Reconstruction')

plt.subplot(223)
plt.plot(xx[20,:], zz[20,:])
plt.plot(xx[20,:], ff[20,:])
plt.plot([0,1], [0,1])
plt.xlim(-.1,1.1); plt.ylim(0,1)


In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
err = np.abs(zz-ff)
cs=plt.contourf(xx,yy,err,levels=np.linspace(0,2,31))
plt.plot(xi, yi, 'ok')
plt.title('Error')
plt.colorbar(cs)